Originally created by Rohan Rao.

Source: [kaggle-kernel](https://www.kaggle.com/rohanrao/calculating-aqi-air-quality-index)

## Calculating AQI (Air Quality Index) in India
![](https://i.imgur.com/GL2BaU4.png)

This notebook provides a workflow for calculating Air Quality Index. The formula is as per CPCB's official AQI Calculator: https://app.cpcbccr.com/ccr_docs/How_AQI_Calculated.pdf


## Preparing data
The dataset used is hourly air quality data (2015 - 2020) from various measuring stations across India: https://www.kaggle.com/rohanrao/air-quality-data-in-india

We'll use one station and compare it with the actual AQI values present in the data to confirm the calculations are correct.


In [2]:
## importing packages
import numpy as np
import pandas as pd

In [4]:
## defining constants
PATH_STATION_HOUR = "/home/ubuntu/datasets/AQI/station_hour.csv"
STATION = "AP001"

In [5]:
## importing data and subsetting the station
df = pd.read_csv(PATH_STATION_HOUR)
df = df[df.StationId == STATION]
df.Datetime = pd.to_datetime(df.Datetime)
df.sort_values("Datetime", inplace = True)


In [6]:
df.columns

Index(['StationId', 'Datetime', 'PM2.5', 'PM10', 'NO', 'NO2', 'NOx', 'NH3',
       'CO', 'SO2', 'O3', 'Benzene', 'Toluene', 'Xylene', 'AQI', 'AQI_Bucket'],
      dtype='object')

In [7]:
df.head(10)

,StationId,Datetime,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket
0,AP001,2017-11-24 17:00:00,60.50,98.00,2.35,30.80,18.25,8.50,0.1,11.85,126.40,0.10,6.10,0.10,NaN,NaN
1,AP001,2017-11-24 18:00:00,65.50,111.25,2.70,24.20,15.07,9.77,0.1,13.17,117.12,0.10,6.25,0.15,NaN,NaN
2,AP001,2017-11-24 19:00:00,80.00,132.00,2.10,25.18,15.15,12.02,0.1,12.08,98.98,0.20,5.98,0.18,NaN,NaN
3,AP001,2017-11-24 20:00:00,81.50,133.25,1.95,16.25,10.23,11.58,0.1,10.47,112.20,0.20,6.72,0.10,NaN,NaN
4,AP001,2017-11-24 21:00:00,75.25,116.00,1.43,17.48,10.43,12.03,0.1,9.12,106.35,0.20,5.75,0.08,NaN,NaN
5,AP001,2017-11-24 22:00:00,69.25,108.25,0.70,18.47,10.38,13.80,0.1,9.25,91.10,0.20,5.02,0.00,NaN,NaN
6,AP001,2017-11-24 23:00:00,67.50,111.50,1.05,12.15,7.30,17.65,0.1,9.40,112.70,0.20,5.60,0.10,NaN,NaN
7,AP001,2017-11-25 00:00:00,68.00,111.00,1.25,14.12,8.50,20.28,0.1,8.90,116.12,0.20,5.55,0.05,NaN,NaN
8,AP001,2017-11-25 01:00:00,73.00,102.00,0.30,14.30,7.90,11.50,0.3,11.80,121.50,0.20,6.60,0.00,NaN,NaN
9,AP001,2017-11-25 02:00:00,81.00,123.00,0.80,24.85,13.88,10.28,0.1,11.62,83.80,0.23,6.77,0.10,NaN,NaN


In [8]:
df.shape

(20816, 16)

## Formula
![](https://i.imgur.com/vQR5Zy0.png)

* The AQI calculation uses 7 measures: **PM2.5, PM10, SO2, NOx, NH3, CO and O3**.
* For **PM2.5, PM10, SO2, NOx and NH3** the average value in last 24-hrs is used with the condition of having at least 16 values.
* For **CO and O3** the maximum value in last 8-hrs is used.
* Each measure is converted into a Sub-Index based on pre-defined groups.
* Sometimes measures are not available due to lack of measuring or lack of required data points.
* Final AQI is the maximum Sub-Index with the condition that at least one of PM2.5 and PM10 should be available and at least three out of the seven should be available.


In [9]:
df["PM10_24hr_avg"] = df["PM10"].rolling(window = 24, min_periods = 16).mean().values
df["PM2.5_24hr_avg"] = df["PM2.5"].rolling(window = 24, min_periods = 16).mean().values
df["SO2_24hr_avg"] = df["SO2"].rolling(window = 24, min_periods = 16).mean().values
df["NOx_24hr_avg"] = df["NOx"].rolling(window = 24, min_periods = 16).mean().values
df["NH3_24hr_avg"] = df["NH3"].rolling(window = 24, min_periods = 16).mean().values
df["CO_8hr_max"] = df["CO"].rolling(window = 8, min_periods = 1).max().values
df["O3_8hr_max"] = df["O3"].rolling(window = 8, min_periods = 1).max().values


## PM2.5 (Particulate Matter 2.5-micrometer)
PM2.5 is measured in ug / m3 (micrograms per cubic meter of air). The predefined groups are defined in the function below:


In [10]:
## PM2.5 Sub-Index calculation
def get_PM25_subindex(x):
    if x <= 30:
        return x * 50 / 30
    elif x <= 60:
        return 50 + (x - 30) * 50 / 30
    elif x <= 90:
        return 100 + (x - 60) * 100 / 30
    elif x <= 120:
        return 200 + (x - 90) * 100 / 30
    elif x <= 250:
        return 300 + (x - 120) * 100 / 130
    elif x > 250:
        return 400 + (x - 250) * 100 / 130
    else:
        return 0

df["PM2.5_SubIndex"] = df["PM2.5_24hr_avg"].apply(lambda x: get_PM25_subindex(x))


## PM10 (Particulate Matter 10-micrometer)
PM10 is measured in ug / m3 (micrograms per cubic meter of air). The predefined groups are defined in the function below:


In [11]:
## PM10 Sub-Index calculation
def get_PM10_subindex(x):
    if x <= 50:
        return x
    elif x <= 100:
        return x
    elif x <= 250:
        return 100 + (x - 100) * 100 / 150
    elif x <= 350:
        return 200 + (x - 250)
    elif x <= 430:
        return 300 + (x - 350) * 100 / 80
    elif x > 430:
        return 400 + (x - 430) * 100 / 80
    else:
        return 0

df["PM10_SubIndex"] = df["PM10_24hr_avg"].apply(lambda x: get_PM10_subindex(x))


## SO2 (Sulphur Dioxide)
SO2 is measured in ug / m3 (micrograms per cubic meter of air). The predefined groups are defined in the function below:

In [12]:
## SO2 Sub-Index calculation
def get_SO2_subindex(x):
    if x <= 40:
        return x * 50 / 40
    elif x <= 80:
        return 50 + (x - 40) * 50 / 40
    elif x <= 380:
        return 100 + (x - 80) * 100 / 300
    elif x <= 800:
        return 200 + (x - 380) * 100 / 420
    elif x <= 1600:
        return 300 + (x - 800) * 100 / 800
    elif x > 1600:
        return 400 + (x - 1600) * 100 / 800
    else:
        return 0

df["SO2_SubIndex"] = df["SO2_24hr_avg"].apply(lambda x: get_SO2_subindex(x))


## NOx (Any Nitric x-oxide)
NOx is measured in ppb (parts per billion). The predefined groups are defined in the function below:


In [13]:
## NOx Sub-Index calculation
def get_NOx_subindex(x):
    if x <= 40:
        return x * 50 / 40
    elif x <= 80:
        return 50 + (x - 40) * 50 / 40
    elif x <= 180:
        return 100 + (x - 80) * 100 / 100
    elif x <= 280:
        return 200 + (x - 180) * 100 / 100
    elif x <= 400:
        return 300 + (x - 280) * 100 / 120
    elif x > 400:
        return 400 + (x - 400) * 100 / 120
    else:
        return 0

df["NOx_SubIndex"] = df["NOx_24hr_avg"].apply(lambda x: get_NOx_subindex(x))


## NH3 (Ammonia)
NH3 is measured in ug / m3 (micrograms per cubic meter of air). The predefined groups are defined in the function below:

In [14]:
## NH3 Sub-Index calculation
def get_NH3_subindex(x):
    if x <= 200:
        return x * 50 / 200
    elif x <= 400:
        return 50 + (x - 200) * 50 / 200
    elif x <= 800:
        return 100 + (x - 400) * 100 / 400
    elif x <= 1200:
        return 200 + (x - 800) * 100 / 400
    elif x <= 1800:
        return 300 + (x - 1200) * 100 / 600
    elif x > 1800:
        return 400 + (x - 1800) * 100 / 600
    else:
        return 0

df["NH3_SubIndex"] = df["NH3_24hr_avg"].apply(lambda x: get_NH3_subindex(x))


## CO (Carbon Monoxide)
CO is measured in ug / m3 (micrograms per cubic meter of air). The predefined groups are defined in the function below:

In [15]:
## CO Sub-Index calculation
def get_CO_subindex(x):
    if x <= 1:
        return x * 50 / 1
    elif x <= 2:
        return 50 + (x - 1) * 50 / 1
    elif x <= 10:
        return 100 + (x - 2) * 100 / 8
    elif x <= 17:
        return 200 + (x - 10) * 100 / 7
    elif x <= 34:
        return 300 + (x - 17) * 100 / 17
    elif x > 34:
        return 400 + (x - 34) * 100 / 17
    else:
        return 0

df["CO_SubIndex"] = df["CO_8hr_max"].apply(lambda x: get_CO_subindex(x))


## O3 (Ozone or Trioxygen)
O3 is measured in ug / m3 (micrograms per cubic meter of air). The predefined groups are defined in the function below:

In [16]:
## O3 Sub-Index calculation
def get_O3_subindex(x):
    if x <= 50:
        return x * 50 / 50
    elif x <= 100:
        return 50 + (x - 50) * 50 / 50
    elif x <= 168:
        return 100 + (x - 100) * 100 / 68
    elif x <= 208:
        return 200 + (x - 168) * 100 / 40
    elif x <= 748:
        return 300 + (x - 208) * 100 / 539
    elif x > 748:
        return 400 + (x - 400) * 100 / 539
    else:
        return 0

df["O3_SubIndex"] = df["O3_8hr_max"].apply(lambda x: get_O3_subindex(x))


## AQI
The final AQI is the maximum Sub-Index among the available sub-indices with the condition that at least one of PM2.5 and PM10 should be available and at least three out of the seven should be available.

There is no theoretical upper value of AQI but its rare to find values over 1000.

The pre-defined buckets of AQI are as follows:
![](https://i.imgur.com/XmnE0rT.png)


In [17]:
## AQI bucketing
def get_AQI_bucket(x):
    if x <= 50:
        return "Good"
    elif x <= 100:
        return "Satisfactory"
    elif x <= 200:
        return "Moderate"
    elif x <= 300:
        return "Poor"
    elif x <= 400:
        return "Very Poor"
    elif x > 400:
        return "Severe"
    else:
        return np.NaN

df["Checks"] = (df["PM2.5_SubIndex"] > 0).astype(int) + \
                (df["PM10_SubIndex"] > 0).astype(int) + \
                (df["SO2_SubIndex"] > 0).astype(int) + \
                (df["NOx_SubIndex"] > 0).astype(int) + \
                (df["NH3_SubIndex"] > 0).astype(int) + \
                (df["CO_SubIndex"] > 0).astype(int) + \
                (df["O3_SubIndex"] > 0).astype(int)

df["AQI_calculated"] = round(df[["PM2.5_SubIndex", "PM10_SubIndex", "SO2_SubIndex", "NOx_SubIndex",
                                 "NH3_SubIndex", "CO_SubIndex", "O3_SubIndex"]].max(axis = 1))
df.loc[df["PM2.5_SubIndex"] + df["PM10_SubIndex"] <= 0, "AQI_calculated"] = np.NaN
df.loc[df.Checks < 3, "AQI_calculated"] = np.NaN

df["AQI_bucket_calculated"] = df["AQI_calculated"].apply(lambda x: get_AQI_bucket(x))
df[~df.AQI_calculated.isna()].head()


,StationId,Datetime,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,...,PM2.5_SubIndex,PM10_SubIndex,SO2_SubIndex,NOx_SubIndex,NH3_SubIndex,CO_SubIndex,O3_SubIndex,Checks,AQI_calculated,AQI_bucket_calculated
16,AP001,2017-11-25 09:00:00,104.00,148.5,1.93,23.00,13.75,9.80,0.1,15.3,...,155.468750,112.562500,14.349219,15.366406,2.844844,5.0,125.911765,7,155.0,Moderate
17,AP001,2017-11-25 10:00:00,94.50,142.0,1.33,16.25,9.75,9.65,0.1,17.0,...,158.970588,113.470588,14.755147,15.179412,2.819412,5.0,153.279412,7,159.0,Moderate
18,AP001,2017-11-25 11:00:00,82.75,126.5,1.47,14.83,9.07,9.70,0.1,15.4,...,159.907407,113.703704,15.004861,14.965972,2.797500,5.0,173.411765,7,173.0,Moderate
19,AP001,2017-11-25 12:00:00,79.00,124.0,5.30,21.15,15.53,9.40,0.1,NaN,...,160.087719,113.824561,15.004861,15.200000,2.773947,5.0,183.529412,7,184.0,Moderate
20,AP001,2017-11-25 13:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,160.087719,113.824561,15.004861,15.200000,2.773947,5.0,183.529412,7,184.0,Moderate


In [18]:
df[~df.AQI_calculated.isna()].AQI_bucket_calculated.value_counts()

Satisfactory    7729
Moderate        5321
Good            3121
Poor            1132
Very Poor        186
Name: AQI_bucket_calculated, dtype: int64

## Verification
Since this exact formula is used for AQI calculated, lets quickly compare it with the actual AQI values present in the raw data.


In [19]:
df_check = df[["AQI", "AQI_calculated"]].dropna()

print("Rows: ", df_check.shape[0])
print("Matched AQI: ", (df_check.AQI == df_check.AQI_calculated).sum())
print("% Match: ", (df_check.AQI == df_check.AQI_calculated).sum() * 100 / df_check.shape[0])


Rows:  17489
Matched AQI:  17489
% Match:  100.0


Matches perfectly. In case of any discrepancy or bug or issue, feel free to comment here or share on the [dataset page](https://www.kaggle.com/rohanrao/air-quality-data-in-india).